In [1]:
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision.transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from models.MNIST import MNIST


In [2]:
# paramaters
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
EPOCHS = 5
LEARNING_RATE = 1e-3


In [3]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])

In [4]:
print("Loading Datas ...")
train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)
print("Loading Datas Finished")

print("train datas of %d" % len(train_loader))
print("test datas of %d" % len(test_loader))

Loading Datas ...
Loading Datas Finished
train datas of 469
test datas of 79


In [5]:
with open('data/MNIST/raw/train-images-idx3-ubyte', 'rb') as f:
    file = f.read()

In [6]:
image = [int(str(item).encode('ascii'), 16) for item in file[16: 16 + 784]]
image_np = np.array(image, dtype=np.uint8).reshape(28, 28, 1)
print(image_np.shape)
cv2.imwrite('test.jpg', image_np)

(28, 28, 1)


True

In [8]:
model = MNIST().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [9]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # 梯度初始化为0
        output = model(data)
        loss = F.cross_entropy(output, target)  # 损失
        pred = output.softmax(dim=1)  # 概率值的最大下标
        loss.backward()
        optimizer.step()
        if batch_index % 100 == 0:
            print("train epoch %d, batch %d, loss %.6f" % (epoch, batch_index, loss.item()))

In [10]:
def evaluate(model, device, test_loader):
    model.eval()
    correct = 0.0
    test_loss = 0.0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target).item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("test average loss %.4f, accuracy %.4f" % (test_loss, 100.0 * correct / len(test_loader.dataset)))

In [11]:
for epoch in range(0, EPOCHS):
    train(model, DEVICE, train_loader, optimizer, epoch)
    evaluate(model, DEVICE, test_loader)

train epoch 0, batch 0, loss 2.305865
train epoch 0, batch 100, loss 0.054794
train epoch 0, batch 200, loss 0.056644
train epoch 0, batch 300, loss 0.107167
train epoch 0, batch 400, loss 0.070186
test average loss 0.0003, accuracy 98.6500
train epoch 1, batch 0, loss 0.063086
train epoch 1, batch 100, loss 0.083722
train epoch 1, batch 200, loss 0.031873
train epoch 1, batch 300, loss 0.032263
train epoch 1, batch 400, loss 0.020221
test average loss 0.0002, accuracy 99.1300
train epoch 2, batch 0, loss 0.012650
train epoch 2, batch 100, loss 0.005618
train epoch 2, batch 200, loss 0.013819
train epoch 2, batch 300, loss 0.006610
train epoch 2, batch 400, loss 0.108925
test average loss 0.0003, accuracy 98.9300
train epoch 3, batch 0, loss 0.013500
train epoch 3, batch 100, loss 0.004102
train epoch 3, batch 200, loss 0.045152
train epoch 3, batch 300, loss 0.038900
train epoch 3, batch 400, loss 0.011184
test average loss 0.0002, accuracy 99.0500
train epoch 4, batch 0, loss 0.00457